## Avaliação Total da Disciplina
### Solução para empresa de logística que opera uma frota de robôs de entrega autônomos

* Contexto:
  * Imagine que você está trabalhando em uma empresa de logística que opera uma frota de robôs de entrega autônomos. Sua tarefa é implementar um algoritmo de aprendizado por reforço para otimizar a entrega de pacotes em um ambiente simulado. Os robôs têm a capacidade de se mover em um ambiente em grade 2D e devem aprender a tomar decisões sobre para onde se mover para entregar pacotes de forma eficiente.

* Objetivo:
  * O objetivo do desafio é implementar um agente de aprendizado por reforço usando ténicas como por exemplo Q-Table, equação de Bellman, ε-greedy e etc, para maximizar o retorno cumulativo ao entregar pacotes no menor tempo possível, considerando um "living penalty" para incentivar o agente a ser eficiente.


* Critérios de Avaliação:
  1.	Implementação correta do ambiente, MDP e Q-Table.
  2.	Implementação do algoritmo de aprendizado por reforço (Q-Learning ou similar).
  3.	Introdução e configuração adequada do "living penalty".
  4.	Eficiência do agente na entrega de pacotes.
  5.	Documentação clara e código bem comentado em Python.

## 1 - Modelagem do ambiente

  * Crie um ambiente 2D simulado, onde o agente pode se mover em um grid. Considere que o ambiente tem obstáculos e pontos de entrega de pacotes.

In [1]:
# @title Importando libs
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Criando um ambiente simulado
class Environment:
    def __init__(self, grid_size, obstacles, delivery_points):
        self.grid_size = grid_size
        self.grid = np.zeros(grid_size, dtype=int)
        self.obstacles = obstacles
        self.delivery_points = delivery_points
        self.agent_position = (0, 0)  # A posição inicial do agente

    def reset(self):
        self.grid = np.zeros(self.grid_size, dtype=int)
        self.agent_position = (0, 0)

        for obs in self.obstacles:
            self.grid[obs] = -1  # Marcando obstáculos no grid

        for delivery_point in self.delivery_points:
            self.grid[delivery_point] = 1  # Marcando pontos de entrega no grid

    def is_valid_move(self, position):
        x, y = position
        return 0 <= x < self.grid_size[0] and 0 <= y < self.grid_size[1] and self.grid[position] != -1

    def take_action(self, action):
        if action == 'up':
            new_position = (self.agent_position[0] - 1, self.agent_position[1])
        elif action == 'down':
            new_position = (self.agent_position[0] + 1, self.agent_position[1])
        elif action == 'left':
            new_position = (self.agent_position[0], self.agent_position[1] - 1)
        elif action == 'right':
            new_position = (self.agent_position[0], self.agent_position[1] + 1)
        else:
            raise ValueError("Ação inválida")

        if self.is_valid_move(new_position):
            self.agent_position = new_position

    def get_state(self):
        return self.agent_position

    def is_at_delivery_point(self):
        return self.grid[self.agent_position] == 1

    def get_reward(self):
        if self.is_at_delivery_point():
            return 10  # Recompensa positiva por entregar um pacote
        else:
            return -1  # Penalização (living penalty) por cada passo

    def is_done(self):
        return all([self.grid[position] == 0 for position in self.delivery_points])

    def print_environment(self):
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                if (i, j) == self.agent_position:
                    print("A", end=' ')
                elif self.grid[i, j] == -1:
                    print("X", end=' ')  # Obstáculo
                elif self.grid[i, j] == 1:
                    print("D", end=' ')  # Ponto de entrega
                else:
                    print("--", end=' ')  # Espaço vazio
            print()

In [3]:
# Exemplo de uso:
grid_size = (10, 10)
obstacles = [(0,5), (1,1), (1,7), (2,4), (2,0), (3,8), (3,3), (4,3), (4,5), (5,1), (5,6), (6,0), (6,5), (7,4), (7,6), (8,2), (8,9), (9,6)]
delivery_points = [(9,9)]
env = Environment(grid_size, obstacles, delivery_points)
env.reset()
env.print_environment()

A -- -- -- -- X -- -- -- -- 
-- X -- -- -- -- -- X -- -- 
X -- -- -- X -- -- -- -- -- 
-- -- -- X -- -- -- -- X -- 
-- -- -- X -- X -- -- -- -- 
-- X -- -- -- -- X -- -- -- 
X -- -- -- -- X -- -- -- -- 
-- -- -- -- X -- X -- -- -- 
-- -- X -- -- -- -- -- -- X 
-- -- -- -- -- -- X -- -- D 


#### Considerações
Aqui criamos a classe `Environment` que representa o ambiente 2D com obstáculos e pontos de entrega de pacotes. O método `reset` é usado para reiniciar o ambiente, o método `take_action` permite que o agente tome ações como ('up', 'down', 'left', 'right'), e outros métodos fornecem informações sobre o estado, recompensa, e se o ambiente foi concluído.

## 2 - Definição do MDP

  * Modele o problema como um MDP, definindo os estados, as ações, as recompensas, a função de transição e o fator de desconto.

In [21]:
# Definição do conjunto de estados (S) e a grade do ambiente
grid_width = grid_size[0]
grid_height = grid_size[1]
num_states = grid_width * grid_height
states = [(i, j) for i in range(grid_width) for j in range(grid_height)]

print(grid_width)
print(grid_height)
print(num_states)
print(states)

In [25]:
# Definição do conjunto de ações (A)
actions = ["up", "down", "left", "right"]
actions

In [6]:
# Definição da função de recompensa (R)
# Vamos usar um dicionário para mapear um par (estado, ação) para uma recompensa
# Por exemplo, se o agente entregar um pacote com sucesso, a recompensa é +10, e -1 por cada ação de movimento
reward_function = {}
for state in states:
    for action in actions:
        reward_function[(state, action)] = -1  # Living penalty
    # Define recompensa alta para estados de entrega bem-sucedida
    if state == (9, 9):
        reward_function[(state, "deliver")] = 10

print(reward_function)

{((0, 0), 'up'): -1, ((0, 0), 'down'): -1, ((0, 0), 'left'): -1, ((0, 0), 'right'): -1, ((0, 1), 'up'): -1, ((0, 1), 'down'): -1, ((0, 1), 'left'): -1, ((0, 1), 'right'): -1, ((0, 2), 'up'): -1, ((0, 2), 'down'): -1, ((0, 2), 'left'): -1, ((0, 2), 'right'): -1, ((0, 3), 'up'): -1, ((0, 3), 'down'): -1, ((0, 3), 'left'): -1, ((0, 3), 'right'): -1, ((0, 4), 'up'): -1, ((0, 4), 'down'): -1, ((0, 4), 'left'): -1, ((0, 4), 'right'): -1, ((0, 5), 'up'): -1, ((0, 5), 'down'): -1, ((0, 5), 'left'): -1, ((0, 5), 'right'): -1, ((0, 6), 'up'): -1, ((0, 6), 'down'): -1, ((0, 6), 'left'): -1, ((0, 6), 'right'): -1, ((0, 7), 'up'): -1, ((0, 7), 'down'): -1, ((0, 7), 'left'): -1, ((0, 7), 'right'): -1, ((0, 8), 'up'): -1, ((0, 8), 'down'): -1, ((0, 8), 'left'): -1, ((0, 8), 'right'): -1, ((0, 9), 'up'): -1, ((0, 9), 'down'): -1, ((0, 9), 'left'): -1, ((0, 9), 'right'): -1, ((1, 0), 'up'): -1, ((1, 0), 'down'): -1, ((1, 0), 'left'): -1, ((1, 0), 'right'): -1, ((1, 1), 'up'): -1, ((1, 1), 'down'): -1, 

In [7]:
# Definição da função de transição (T)
# Vamos criar uma função de transição determinística para este exemplo
# Ela mapeia (estado, ação) para o próximo estado
transition_function = {}
for state in states:
    for action in actions:
        if action == "up":
            next_state = (max(state[0] - 1, 0), state[1])
        elif action == "down":
            next_state = (min(state[0] + 1, grid_width - 1), state[1])
        elif action == "left":
            next_state = (state[0], max(state[1] - 1, 0))
        elif action == "right":
            next_state = (state[0], min(state[1] + 1, grid_height - 1))
        transition_function[(state, action)] = next_state

print(transition_function)

{((0, 0), 'up'): (0, 0), ((0, 0), 'down'): (1, 0), ((0, 0), 'left'): (0, 0), ((0, 0), 'right'): (0, 1), ((0, 1), 'up'): (0, 1), ((0, 1), 'down'): (1, 1), ((0, 1), 'left'): (0, 0), ((0, 1), 'right'): (0, 2), ((0, 2), 'up'): (0, 2), ((0, 2), 'down'): (1, 2), ((0, 2), 'left'): (0, 1), ((0, 2), 'right'): (0, 3), ((0, 3), 'up'): (0, 3), ((0, 3), 'down'): (1, 3), ((0, 3), 'left'): (0, 2), ((0, 3), 'right'): (0, 4), ((0, 4), 'up'): (0, 4), ((0, 4), 'down'): (1, 4), ((0, 4), 'left'): (0, 3), ((0, 4), 'right'): (0, 5), ((0, 5), 'up'): (0, 5), ((0, 5), 'down'): (1, 5), ((0, 5), 'left'): (0, 4), ((0, 5), 'right'): (0, 6), ((0, 6), 'up'): (0, 6), ((0, 6), 'down'): (1, 6), ((0, 6), 'left'): (0, 5), ((0, 6), 'right'): (0, 7), ((0, 7), 'up'): (0, 7), ((0, 7), 'down'): (1, 7), ((0, 7), 'left'): (0, 6), ((0, 7), 'right'): (0, 8), ((0, 8), 'up'): (0, 8), ((0, 8), 'down'): (1, 8), ((0, 8), 'left'): (0, 7), ((0, 8), 'right'): (0, 9), ((0, 9), 'up'): (0, 9), ((0, 9), 'down'): (1, 9), ((0, 9), 'left'): (0, 

#### Neste código modelamos um `MDP`, especificando:

* `(S)` - O conjunto de estados
* `(A)` - O conjunto de ações
* `(R)` - A função de recompensa
* `(T)` - A função de transição (que descreve a probabilidade de transição de um estado para o outro após uma ação)


## 3 - Implementação da Q-Table

  * Crie uma Q-Table para representar o valor estimado de cada par (estado, ação).

In [8]:
# Cria uma matriz de zeros com dimensões (número de estados, número de ações)
num_states = len(states)
num_actions = len(actions)

print(num_states)
print(num_actions)

100
4


In [9]:
# Inicialização Q-Table com zeros
q_table = np.zeros((num_states, num_actions))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0.

#### Considerações
 
* Criamos uma Q-Table inicializada com zeros. Cada linha representa um estado e cada coluna uma ação.
* O agente irá atualizar os valores nessa tabela durante o treinamento usando o algoritmo de aprendizado por reforço.

## 4 - Implementação do agente

  * Desenvolva um agente de aprendizado por reforço que utiliza a equação de Bellman para atualizar a Q-Table com

In [10]:
# Parâmetros do algoritmo Q-Learning
learning_rate = 0.1  # Taxa de aprendizado
learning_rate

0.1

In [11]:
discount_factor = 0.9  # Fator de desconto
discount_factor

0.9

In [12]:
exploration_prob = 0.2  # Probabilidade de explorar (ε-greedy)
exploration_prob

0.2

In [13]:
# Treinamento do agente
num_episodes = 1000  # Número de episódios de treinamento
for episode in range(num_episodes):
    state = (0, 0)  # Estado inicial

    while state != (9,9):  # Enquanto o agente não entregar o pacote com sucesso
        # Escolha a ação com base na política ε-greedy
        if np.random.rand() < exploration_prob:
            action = np.random.choice(actions)
        else:
            action = actions[np.argmax(q_table[states.index(state)])]

        # Realize a ação e observe o novo estado e a recompensa
        next_state = transition_function[(state, action)]
        reward = reward_function[(state, action)]

        # Atualize a Q-Table usando a equação de Bellman
        q_table[states.index(state), actions.index(action)] = (1 - learning_rate) * q_table[states.index(state), actions.index(action)] + learning_rate * (reward + discount_factor * np.max(q_table[states.index(next_state)]))

        state = next_state  # Atualize o estado para o próximo
        # print(state, action, reward, q_table)

In [14]:
state

(9, 9)

#### No código acima concluímos os passos:
* Definição das regras 
    * learning_rate
    * discount_factor
    * exploration_prob (ε-greedy)
* Treinamos o agente
* Atualização dos valores da Q-Table

## 5 - Living Penalty

  * Introduza um "living penalty" para penalizar o agente por gastar muito tempo no ambiente. Isso deve incentivar o agente a encontrar a rota mais eficiente para entregar os pacotes.

In [15]:
# Definição da função de recompensa com "living penalty"
reward_function = {}
for state in states:
    for action in actions:
        reward = -1  # Penalização padrão por cada ação de movimento
        # Ajuste a penalização para estados próximos aos obstáculos para incentivar o agente a evitá-los
        if state in [(0,5), (1,1), (1,7), (2,4), (2,0), (3,8), (3,3), (4,3), (4,5), (5,1), (5,6), (6,0), (6,5), (7,4), (7,6), (8,2), (8,9), (9,6)] and action != "deliver":
            reward -= 5
        reward_function[(state, action)] = reward
    # Define recompensa alta para estados de entrega bem-sucedida
    if state == (9, 9):
        reward_function[(state, "deliver")] = 10

print(reward_function)

{((0, 0), 'up'): -1, ((0, 0), 'down'): -1, ((0, 0), 'left'): -1, ((0, 0), 'right'): -1, ((0, 1), 'up'): -1, ((0, 1), 'down'): -1, ((0, 1), 'left'): -1, ((0, 1), 'right'): -1, ((0, 2), 'up'): -1, ((0, 2), 'down'): -1, ((0, 2), 'left'): -1, ((0, 2), 'right'): -1, ((0, 3), 'up'): -1, ((0, 3), 'down'): -1, ((0, 3), 'left'): -1, ((0, 3), 'right'): -1, ((0, 4), 'up'): -1, ((0, 4), 'down'): -1, ((0, 4), 'left'): -1, ((0, 4), 'right'): -1, ((0, 5), 'up'): -6, ((0, 5), 'down'): -6, ((0, 5), 'left'): -6, ((0, 5), 'right'): -6, ((0, 6), 'up'): -1, ((0, 6), 'down'): -1, ((0, 6), 'left'): -1, ((0, 6), 'right'): -1, ((0, 7), 'up'): -1, ((0, 7), 'down'): -1, ((0, 7), 'left'): -1, ((0, 7), 'right'): -1, ((0, 8), 'up'): -1, ((0, 8), 'down'): -1, ((0, 8), 'left'): -1, ((0, 8), 'right'): -1, ((0, 9), 'up'): -1, ((0, 9), 'down'): -1, ((0, 9), 'left'): -1, ((0, 9), 'right'): -1, ((1, 0), 'up'): -1, ((1, 0), 'down'): -1, ((1, 0), 'left'): -1, ((1, 0), 'right'): -1, ((1, 1), 'up'): -6, ((1, 1), 'down'): -6, 

## 6 - Treinamento e avaliação

  * Treine o agente usando um algoritmo de aprendizado por reforço, como o Q-Learning, e avalie seu desempenho em termos de eficiência na entrega de pacotes.

In [16]:
num_episodes = 100  # Número de episódios para avaliação
num_episodes

100

In [17]:
success_count = 0  # Contador de entregas bem-sucedidas
success_count

0

In [18]:
total_delivery_time = 0  # Tempo total de entrega
total_delivery_time

0

In [20]:
for episode in range(num_episodes):
    state = (0, 0)  # Estado inicial
    delivery_time = 0

    while state != (9, 9):  # Enquanto o agente não entregar o pacote com sucesso
        # Escolha a ação com base na política aprendida (não ε-greedy durante a avaliação)
        action = actions[np.argmax(q_table[states.index(state)])]

        # Realize a ação e observe o novo estado
        next_state = transition_function[(state, action)]

        state = next_state  # Atualize o estado para o próximo
        delivery_time += 1

    if state == (9, 9):  # Entrega bem-sucedida
        success_count += 1
        total_delivery_time += delivery_time

KeyboardInterrupt: 

In [ ]:
# Cálculo das estatísticas
average_delivery_time = total_delivery_time / success_count
success_rate = success_count / num_episodes

In [ ]:
print("Tempo médio de entrega:", average_delivery_time)
print("Taxa de sucesso na entrega:", success_rate)